In [2]:
import pandas as pd

In [3]:
from pathlib import Path

import sequence_sensei
import numpy as np

In [4]:
import tqdm

In [5]:
import axelrod as axl

In [6]:
import keras

Using TensorFlow backend.
//anaconda3/envs/rnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/envs/rnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/envs/rnn/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/envs/rnn/lib/python3.6/site-packages/tensorflow/py

In [7]:
axl.__version__

'4.2.0'

In [8]:
axl.Action(1)

C

**Outputs**

In [9]:
dfs = []
for file in Path("best_responses/").glob("*.csv"):
    dfs.append(pd.read_csv(file, index_col=0,))

In [13]:
df = pd.read_csv('raw_data/Alternator_nan/main.csv')

In [27]:
table = df[(df["generation"] == 2000)].reset_index(drop=True).head()
table

,opponent,seed,num. of generations,bottleneck,mutation probability,half size population,generation,index,score,gene_0,...,gene_195,gene_196,gene_197,gene_198,gene_199,gene_200,gene_201,gene_202,gene_203,gene_204
0,Alternator,NaN,2000,10,0.01,10,2000,0,3.009756,0,...,0,0,0,0,0,0,0,0,0,0
1,Alternator,NaN,2000,10,0.01,10,2000,1,3.009756,0,...,0,0,0,0,0,0,0,0,0,0
2,Alternator,NaN,2000,10,0.01,10,2000,2,3.009756,0,...,0,0,0,0,0,0,0,0,0,0
3,Alternator,NaN,2000,10,0.01,10,2000,3,3.009756,0,...,0,0,0,0,0,0,0,0,0,0
4,Alternator,NaN,2000,10,0.01,10,2000,4,3.009756,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
file = open('/Users/storm/rsc/Thesis/src/chapters/06/output_table.tex', "w")
file.write(table.to_latex())
file.close()

In [12]:
df = pd.concat(dfs).reset_index(drop=True)

In [13]:
df = df.drop(columns=['index'])

In [14]:
df = df[~(df['opponent'] == 'Go By Majority')]
df = df[~(df['opponent'] == 'Go By Majority 5')]
df = df[~(df['opponent'] == 'Go By Majority 10')]
df = df[~(df['opponent'] == 'Go By Majority 20')]
df = df[~(df['opponent'] == 'Go By Majority 40')]

In [15]:
len(df)

5258

In [17]:
len(df['opponent'].unique())

192

In [19]:
file = open('notes/unique_strategies_opponents_best_sequences.tex', "w")
file.write("{}".format(len(df['opponent'].unique())))
file.close()

**Inputs**

In [18]:
import axelrod as axl

In [19]:
strategies = {strategy().name: strategy() for strategy in axl.all_strategies}

In [20]:
names = df['opponent'].unique()

In [21]:
len(strategies)

235

In [22]:
stochastic = []
for i, opponent in enumerate(tqdm.tqdm_notebook(df['opponent'].unique())):
    
    majority = 'Majority'
    if majority in opponent:
        name = opponent.split(majority)[0] + 'Majority:' + opponent.split(majority)[-1]
        if name[-1] == ":":
            name = name[:-1]
        opponent = strategies[name]
    
    else:
        opponent = strategies[opponent.split(':')[0]]
        
    stochastic.append(opponent.classifier['stochastic'])

In [25]:
len(stochastic) - sum(stochastic)

135

In [24]:
sum(stochastic)

62

In [17]:
opponents_moves = []
for i, opponent in enumerate(tqdm.tqdm_notebook(df['opponent'])):
    
    majority = 'Majority'
    if majority in opponent:
        name = opponent.split(majority)[0] + 'Majority:' + opponent.split(majority)[-1]
        if name[-1] == ":":
            name = name[:-1]
        opponent = strategies[name]
    
    else:
        opponent = strategies[opponent.split(':')[0]]
    
    sequence = sequence_sensei.get_sequence_str(df.iloc[i].values[-205:])
    cycler = axl.Cycler(sequence)

    match = axl.Match([opponent, cycler], turns=205)
    _ = match.play()
    
    moves_opp, moves_cycler = zip(*match.result)
    
    assert sequence_sensei.get_sequence_str(moves_cycler) == sequence
    
    opponents_moves.append((opponent,) + moves_opp)

**Transform Data for RNN**

In [18]:
sequences = [[moves[0]] + [move.value for move in moves[1:]] for moves in opponents_moves]

In [19]:
data = pd.DataFrame(sequences)

In [20]:
data.columns = df.columns

In [21]:
data.to_csv('data/sequences.csv'), df.to_csv('data/targets.csv')

(None, None)

**Transform the data**

In [22]:
sequences = pd.read_csv('data/sequences.csv', index_col=0)
targets = pd.read_csv('data/targets.csv', index_col=0)

In [23]:
inputs = sequences.replace({0: 2})

In [24]:
inputs = inputs.drop(columns=["opponent", "gene_204"]).values
outputs = targets.drop(columns=["opponent", "gene_0"]).values

In [25]:
max_length = len(inputs[0])

In [26]:
prep_X_train = []
prep_y_train = []

for i, sequence in enumerate(inputs):
    assert len(sequence) == max_length
    for histories in range(1, max_length + 1):
        prep_X_train.append(sequence[:histories])
        prep_y_train.append(outputs[i][histories - 1])        

In [27]:
padded_X_train = keras.preprocessing.sequence.pad_sequences(prep_X_train, maxlen=max_length, padding='post') 

In [28]:
padded_X = pd.DataFrame(padded_X_train, columns=[i for i in range(max_length)])

In [29]:
padded_X.to_csv('data/padded_inputs_classification.csv')

In [30]:
y = pd.DataFrame(prep_y_train, columns=['target'])

In [31]:
y.to_csv('data/classification_output.csv')

In [32]:
len(y), len(padded_X)

(1122612, 1122612)